# Implement the ansatz

In [1]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, CXGate, CZGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [2]:
def anzats_circ(thetas, phis, D1, D2, in_state):
    qr = QuantumRegister(4, name="q")
    qc = QuantumCircuit(qr)
    qc.initialize(in_state)
    for d in range(D1):
        qc.append(RXGate(phis[0]), [qr[2]])
        qc.append(RXGate(phis[1]), [qr[3]])
        
        qc.append(RZGate(phis[2]), [qr[2]])
        qc.append(RZGate(phis[3]), [qr[3]])
        
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
        
    for d in range(D2):
        qc.append(RXGate(thetas[0]), [qr[0]])
        qc.append(RXGate(thetas[1]), [qr[1]])
        qc.append(RXGate(thetas[2]), [qr[2]])
        qc.append(RXGate(thetas[3]), [qr[3]])
        
        qc.append(RZGate(thetas[4]), [qr[0]])
        qc.append(RZGate(thetas[5]), [qr[1]])
        qc.append(RZGate(thetas[6]), [qr[2]])
        qc.append(RZGate(thetas[7]), [qr[3]])
        
        qc.append(CZGate(), [qr[0], qr[1]])
        qc.append(CZGate(), [qr[1], qr[2]])
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    
    qc.append(RXGate(thetas[0]), [qr[0]])
    qc.append(RXGate(thetas[1]), [qr[1]])
    qc.append(RXGate(thetas[2]), [qr[2]])
    qc.append(RXGate(thetas[3]), [qr[3]])

    qc.append(RZGate(thetas[4]), [qr[0]])
    qc.append(RZGate(thetas[5]), [qr[1]])
    qc.append(RZGate(thetas[6]), [qr[2]])
    qc.append(RZGate(thetas[7]), [qr[3]])
    
    #qc.measure_all()
    
    return qc

# Choose k orthogonal states(computational basis)

In [3]:
import numpy as np
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]


# Generating the hamiltonians(for 4 qubits)

In [4]:
from qiskit.opflow import X, Z, I, H

# for convinience let all coefficients ai and Jij be 0.5
H_transverse_ising = 0.5*((I^I^I^X) + (I^I^X^I) + (I^X^I^I) + (X^I^I^I) + \
                          (Z^Z^I^I) + (Z^I^Z^I) + (Z^I^I^Z) + (I^Z^Z^I) + \
                          (I^Z^I^Z) + (I^I^Z^Z))

# TODO: change this to a 4 qubits Hamiltonian
H2_molecule_Hamiltonian = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

# Calculating the target function

In [5]:
import matplotlib
from qiskit import assemble, Aer
from qiskit import *
from qiskit.visualization import plot_histogram
import math

sim = Aer.get_backend('aer_simulator')

def convert_string_to_index(string):
    return int(string, 2)

def convert_result_to_state(result):
    total_meas = 0
    
    for k in result.keys():
        total_meas += result[k]
        amount_of_qubits = len(k)

    state = np.zeros(2**amount_of_qubits)
    
    for k in result.keys():
        index = convert_string_to_index(k)
        state[index] = result[k]/total_meas
        
    return state

def calc_ising_avg(qc):
    avg = 0
    for i in range(4):
        qc_tmp = qc.copy()
        qc_tmp.append(H, [i])

def calc_molecular_avg(qc):
    pass

def calc_target_func1(theta, basis, D1, D2, Ham):
    target_func = 0
    for j in basis:
        qc = anzats_circ(theta[0:8], theta[8:], D1, D2, j)
#         print(qc.draw())
#         qobj = assemble(qc)
#         result = sim.run(qobj).result().get_counts()
#         state = convert_result_to_state(result)
#         dagger_state = np.matrix(state)
#         state = dagger_state.getH()
#         state = np.array(state)
#         dagger_state = np.array(dagger_state)
        
#         TODO: calculate this with estimation with the Ham pauli strings
        if Ham == "Ising Model":
            product = calc_ising_avg(qc)
        else:
            product = calc_molecular_avg(qc)
#         product = np.matmul(Ham, state)
#         product = np.matmul(dagger_state, product)
                
        target_func += product
        
    return target_func



In [7]:
# JUST TO RUN AND CHECK
k = 4
basis = get_k_basis(k,16)
D1 = 1
D2 = 1
Ham = "Ising Model"
thetas = np.zeros(12)
print(thetas)
calc_target_func1(thetas, basis, D1, D2, Ham)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'

# Setting the rapping objective function to be sent to BFGS:

In [8]:
def objective_func1(thetas):
    k = 4
    basis = get_k_basis(k,16)
    D1 = 1
    D2 = 1
    Ham = np.identity(16)
#     Ham = H_transverse_ising
    target_func = calc_target_func1(thetas, basis, D1, D2, Ham)
    print("target func:")
    print(target_func[0][0])
    return target_func[0][0]


# Sending the target function to the BFGS optimazer 

In [9]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

point, value, nfev = bfgs_optimizer.optimize(12,objective_func1,initial_point=np.zeros(12))
print(point)
print("---point---")
print(value)
print("---value---")
print(nfev)
print("---nfev---")

thetas_opt = point


C:\Users\hadar\AppData\Local\Temp\ipykernel_40520\2862467496.py:50: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if Ham == "Ising Model":


TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'

# Construct another parametrized quantum circuit


In [ ]:
def anzats_circ2(thetas,D1, D2, in_state):
    qr = QuantumRegister(4, name="q")
    #cr = ClassicalRegister(4, name='c')
    qc = QuantumCircuit(qr)
    qc.initialize(in_state)